In [1]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Annotated,Literal
from openai import AzureOpenAI 
from dotenv import load_dotenv
import os
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_openai import AzureChatOpenAI  
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

In [2]:
load_dotenv()

# ✅ Step 3: Fetch environment variables from the system
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    api_version=azure_api_version
)

# ✅ Use LangChain Azure wrapper — supports structured output
model = AzureChatOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=azure_api_key,
    api_version=azure_api_version,
    azure_deployment=azure_deployment,
    temperature=0,
)


In [3]:
from langchain_core.messages import BaseMessage, HumanMessage
class ChatState(TypedDict):

    messages: Annotated[list[BaseMessage], add_messages]

In [4]:
def chat_node(state: ChatState):

    # take user query from state
    messages = state['messages']

    # send to llm
    response = model.invoke(messages)

    # response store state
    return {'messages': [response]}

In [5]:
checkpointer=MemorySaver()
graph = StateGraph(ChatState)

# add nodes
graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile(checkpointer=checkpointer)

In [6]:
# ----------- Chat Loop -----------
thread_id = "1"

print("AI Chatbot Ready! Type 'exit' to quit.\n")

while True:
    user_message = input("You: ")

    if user_message.strip().lower() in ["exit", "quit", "bye"]:
        print("AI: Goodbye! 👋")
        break

    config = {"configurable": {"thread_id": thread_id}}

    response = chatbot.invoke(
        {"messages": [HumanMessage(content=user_message)]},
        config=config
    )

    # Last assistant message
    ai_reply = response["messages"][-1].content

    print(f"\n--- Conversation Turn ---")
    print(f"Human: {user_message}")
    print(f"AI   : {ai_reply}\n")

AI Chatbot Ready! Type 'exit' to quit.


--- Conversation Turn ---
Human: hi my name is laxmi kant
AI   : Hello, Laxmi Kant! 😊 How can I assist you today?


--- Conversation Turn ---
Human: tell me what is factorial of 4
AI   : The factorial of a number is the product of all positive integers from 1 to that number. 

For **4**, the factorial is calculated as:

\[
4! = 4 \times 3 \times 2 \times 1 = 24
\]

So, the factorial of **4** is **24**. 😊


--- Conversation Turn ---
Human: sorry can you remember my name
AI   : Of course, Laxmi Kant! 😊 I'll remember your name during this conversation. How can I assist you further?

AI: Goodbye! 👋
